In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Dropout, Flatten ,GlobalMaxPooling1D,GRU


# Load the dataset
file_path = 'Truth_Seeker_Model_Dataset.csv'
file_path2 = 'Features_For_Traditional_ML_Techniques.csv'
file_path3 = 'Downloads/Truth_Seeker_Model_Dataset_With_TimeStamps 1.xlsx'
df = pd.read_csv(file_path)

df2 = pd.read_csv(file_path2)

df3 = pd.read_excel(file_path3)

df = df.drop(columns=["Unnamed: 0"])

df2 = df2.drop(columns=["Unnamed: 0"])

df3 = df3.drop(columns=["Unnamed: 0"])

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

#df2 = df2.sample(frac=1, random_state=42).reset_index(drop=True)

#df3 = df3.sample(frac=1, random_state=42).reset_index(drop=True)


In [4]:



# # this will download the BERT Tokenizer
# bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  
# # this will download the BERT Trained Model
# # output_hidden_states=False, as we are training & not interested in output state.
# config = BertConfig.from_pretrained("bert-base-uncased",output_hidden_states=False) # dropout=0.2, attention_dropout=0.2
# bert_model = TFBertModel.from_pretrained('bert-base-uncased', config=config)


In [5]:


# import re
# from tqdm import tqdm


# def clean_text(temp):
#     temp=re.sub("@\S+", " ", temp)
#     temp=re.sub("https*\S+", " ", temp)
#     temp=re.sub("#\S+", " ", temp)
#     temp=re.sub("\'\w+", '', temp)
#     temp=re.sub(r'\w*\d+\w*', '', temp)
#     temp=re.sub('\s{2,}', " ", temp)
#     return temp.strip()

# shuffled_df['tweet_clean'] = shuffled_df['tweet'].apply(clean_text)
# sentences = shuffled_df['tweet_clean'] + shuffled_df['statement'] + shuffled_df['author'] + shuffled_df['manual_keywords'] + shuffled_df['5_label_majority_answer'] + shuffled_df['3_label_majority_answer'] 

In [6]:


# import numpy as np


# input_ids=[]
# attention_masks=[]

# for sent in tqdm(sentences):
#     bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =128,pad_to_max_length = True,return_attention_mask = True)
#     input_ids.append(bert_inp['input_ids'])
#     attention_masks.append(bert_inp['attention_mask'])

# input_ids=np.asarray(input_ids)
# attention_masks=np.array(attention_masks)
# labels = np.array(shuffled_df['BinaryNumTarget'])


In [7]:


#features = np.concatenate((input_ids, attention_masks), axis=-1)

In [8]:
# # Preprocess the text data
statements = df['statement'].astype(str).values
tweets = df['tweet'].astype(str).values
author = df['author'].astype(str).values
# fivelabel = batch1['5_label_majority_answer'].astype(str).values
# threelabel = batch1['3_label_majority_answer'].astype(str).values
labels = df['BinaryNumTarget'].astype(int).values

# # Combine statements and tweets into one feature
# #combined_text = [s + " " + t for s, t in zip(statements, tweets)]

combined_text = [
     f"{s} {t} [SEP] Author: {a}"
     for s, t, a in zip(statements, tweets, author)
 ]


In [9]:
# Tokenize the text data
max_num_words = 500
max_sequence_length = 28

tokenizer = Tokenizer(num_words=max_num_words, oov_token="<OOV>")
tokenizer.fit_on_texts(combined_text)
sequences = tokenizer.texts_to_sequences(combined_text)
word_index = tokenizer.word_index

#Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [10]:
# from sklearn.preprocessing import MinMaxScaler

# # Normalize numerical user features
# scaler = MinMaxScaler()
# user_features_normalized = scaler.fit_transform(shuffled_df[['hashtags', 'URLs', 'PERSON_percentage', 
#                                                       'LANGUAGE_percentage', 'pronouns', 'exclamation','past_verbs', 'adjectives','capitals', 'digits',
#                                                         'adpositions','present_verbs','ampersand','MONEY_percentage','PERCENT_percentage',
#                                                          'QUANTITY_percentage','LOC_percentage','WORK_OF_ART_percentage','NORP_percentage','EVENT_percentage',
#                                                         'unique_count','CARDINAL_percentage','LAW_percentage','listed_count','followers_count',
#                                                          'favourites_count','normalize_influence','favourites']])

# # Convert normalized features to TensorFlow tensors
# user_features_tensor = tf.convert_to_tensor(user_features_normalized, dtype=tf.float32)

# # Combine textual and numerical features
# combined_features = tf.concat([features, user_features_tensor], axis=1)


In [11]:
#import numpy as np

# # Ensure combined_features and labels are NumPy arrays
#combined_features_np = np.array(combined_features)  # Convert to NumPy


In [12]:




# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)


# Tokenize and pad the input data
# X_train_padded = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
# X_test_padded = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')

# Build and compile the hybrid model
embedding_dim = 16

model = Sequential()
model.add(Embedding(input_dim=max_num_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.1))
model.add(GRU(16, return_sequences=True, dropout=0.2, recurrent_dropout=0))
model.add(GlobalMaxPooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
batch_size = 32
epochs = 10



#history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=2)

history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    
)

# Predict on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 16)            8000      
                                                                 
 spatial_dropout1d (SpatialD  (None, 28, 16)           0         
 ropout1D)                                                       
                                                                 
 gru (GRU)                   (None, 28, 16)            1632      
                                                                 
 global_max_pooling1d (Globa  (None, 16)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dropout (Dropout)           (None, 16)                0